In [ ]:
import pandas as pd
from pandas.io.json import json_normalize
from pandas import DataFrame
import os
from geopy import geocoders
from geopy.geocoders import GoogleV3
from config import gkey

import geopandas as gpd
from shapely import wkt
from shapely.wkt import loads
import re

In [ ]:
# Pull in geojson address data from openaddresses
fname = "resources/us_il_city_of_evanston-addresses-city.geojson"

# read file to dataframe
gdf = gpd.read_file(fname)

simpledec = re.compile(r"\d*\.\d+")
def mround(match):
    return "{:.5f}".format(float(match.group()))

gdf.geometry = gdf.geometry.apply(lambda x: loads(re.sub(simpledec, mround, x.wkt)))
gdf

In [ ]:
# Verify/Make pandas dataframe
addressdf = pd.DataFrame(gdf)

# Change 'geometry' column datatype from geometry to object
addressdf['geometry'] = addressdf.geometry.apply(lambda x: wkt.dumps(x, rounding_precision=5))


In [ ]:
# Check dataframe
addressdf

In [ ]:
# Combine address columns to one column
# addressdf['address'] = addressdf['street'].str.cat(addressdf['city'].str.cat(addressdf['region'],sep=" ")

# Remove extra strings from the latitude and longitude in the geometry column
addressdf['geometry'] = addressdf['geometry'].str.replace('POINT ', '').str.replace('(', '').str.replace(')', '')
addressdf

In [ ]:
# Remove un-needed columns
addressdf.drop(['unit', 'district'],axis=1,inplace=True)
addressdf

In [ ]:

# fltraddress = addressdf[addressdf['addresses'].str.lower().str.contains('^6')]
# fltraddress

In [ ]:
# adds_sml = pd.read_csv("static/data/sample/adds_sml.csv")
def format_coordinates(x):
    coordinates = x.split(' ')
    lat = str(coordinates[1])
    long = str(coordinates[0])
    formatted = lat + ', ' + long
    return formatted
addressdf['geometry'] = addressdf['geometry'].apply(format_coordinates)
addressdf


In [ ]:
addressdf['city'] = addressdf['city'].apply(lambda x: f"{x},")
addressdf

In [ ]:
addressdf = addressdf.assign(addresses = addressdf.number.astype(str) + " " +\
  addressdf.street.astype(str) + " " + addressdf.city.astype(str) + " " + \
  addressdf.region.astype(str) + " " +addressdf.postcode.astype(str))
addressdf

In [ ]:
adds_sml = addressdf.sample(n = 100) 
adds_sml

In [ ]:
import numpy as np
address_list = adds_sml['addresses'].to_numpy()
np.savetxt("static/data/raw/sample_address_list.txt", address_list, fmt='%5s')

In [ ]:
# GRAB A SERIES OF STREET VIEW IMAGES FROM latlong PROVIDED
import json
# Import google_streetview for the api module
import google_streetview.api
import time
import glob
import streetview
import itertools 


In [ ]:
#import os and urllib modules
#os for file path creation
#urllib for accessing web content
import urllib.request

 
#this is the first part of the streetview, url up to the address, this url will return a 600x600px image
#pre="https://maps.googleapis.com/maps/api/streetview?size=600x600&amp;location="
pre="https://maps.googleapis.com/maps/api/streetview?size=600x600&location="
 
#this is the second part of the streetview url, the text variable below, includes the path to a text file containing one address per line
#the addresses in this text file will complete the URL needed to return a streetview image and provide the filename of each streetview image
text="static/data/raw/sample_address_list.txt"
 
#this is the third part of the url, needed after the address
#this is my API key, please replace the one below with your own (google 'google streetview api key'), thanks!
suf=f"&key={gkey}&fov=60"
 
#this is the directory that will store the streetview images
#this directory will be created if not present
dir=r"static/data/raw/locs/"
 
#checks if the dir variable (output path) above exists and creates it if it does not
if not os.path.exists(dir):
    os.makedirs(dir)
 
#opens the address list text file (from the 'text' variable defined above) in read mode ("r")
with open(text,"r") as text_file:
  #the variable 'lines' below creates a list of each address line in the source 'text' file
  lines = [line.rstrip('\n') for line in open(text)]
#   print("THE CONTENTS OF THE TEXT FILE:\n"+str(lines))
  #start a loop through the 'lines' list
  for line in lines:
    #string clean-up to get rid of commas in the url and filename
    ln = line.replace(" " , "+")
#     print("CLEANED UP ADDRESS LINE:\n"+ln)
    # creates the url that will be passed to the url reader, this creates the full, valid, url that will return a google streetview image for each address in the address text file
    URL = pre+ln+suf
#     print("URL FOR STREETVIEW IMAGE:\n"+URL)
    #creates the filename needed to save each address's streetview image locally
    filename = os.path.join(dir,ln+".jpg")
#     print("OUTPUT FILENAME:\n"+filename)
    #you can run this up to this line in the python command line to see what each step does
    #final step, fetches and saves the streetview image for each address using the url created in the previous steps
    urllib.request.urlretrieve(URL, filename)

In [ ]:
# adds_sml.to_csv (r'static/data/sample/adds_sml.csv', index = True, header=True)

In [ ]:
# # create a params dict that will be updated with new city each iteration
# params = {
#     'size': '640x640', # max 640x640 pixels
#     'heading': '90',
#     'location': '',
#     'pitch': '-0.76',
#     'fov': 120,
#     'key': gkey
# }

# # Loop through the cities_pd and run a lat/long search for each city
# geometry_list = adds_sml['geometry'].to_list()
# i = 0
# for latlong in geometry_list[50:60]:
    
#     # update address key value
#     params['location'] = latlong
    
#     # Create a results object
# #     results = google_streetview.api.results(params)
#     results = google_streetview.api.results([params])
    
    
     

#     # Download images to directory 'downloads'
#     results.download_links(f"static/data/raw/geo{i}",)
#     old_file_name = f"static/data/raw/geo{i}/gsv_0.jpg"
#     new_file_name = f"static/data/raw/geo{i}/gsv_{i}.jpg"

#     os.rename(old_file_name, new_file_name)
    
#     #pull .jpeg into main folder rename .jpeg{i}
    
#     i += 1
    
#     # Wait for 1 second
#     #time.sleep(1)

In [ ]:
# # *** ORIGINAL PULL THAT WORKS ***
# # Define parameters for street view api
# params = [{
#     'size': '600x300', # max 640x640 pixels
#     'location': '41.9295582,-87.7159139',
#     'heading': '90',
#     'pitch': '-0.76',
#     'key': gkey
# }]

# # Create a results object
# results = google_streetview.api.results(params)

# # Download images to directory 'downloads'
# results.download_links('static/data/raw')

In [ ]:
## PRACTICING IMAGE PRE-PROCESSING

# # load and show an image with Pillow

# # load the image
# image = Image.open('static/data/test/brickornot.jpg')
# # summarize some details about the image
# print(image.format)
# print(image.mode)
# print(image.size)
# # show the image
# image.show()

In [ ]:
'''# load all images in a directory
loaded_images = list()
for filename in listdir('images'):
	# load image
	img_data = image.imread('images/' + filename)
	# store loaded image
	loaded_images.append(img_data)
	print('> loaded %s %s' % (filename, img_data.shape))'''


In [ ]:
%matplotlib inline
import PIL
from PIL import Image
from os import listdir
from matplotlib import image
from matplotlib import pyplot

# example of saving a grayscale version of a loaded image
from PIL import Image
# load the image
image = Image.open('static/data/raw/gsv_0.jpg')
# convert the image to grayscale
gs_image = image.convert(mode='L')
# report the size of the image
print(image.size)
# create a thumbnail and preserve aspect ratio
image.thumbnail((100,100))
# report the size of the thumbnail
print(image.size)
# save in jpeg format
gs_image.save('static/data/test_convert/gsv_0.jpg')

In [ ]:
# load the image again and show it
image2 = Image.open('static/data/test_convert/gsv_0.jpg')
# show the image
image2.show()

In [ ]:
'''# save as PNG format
image.save('opera_house.png', format='PNG')
# load the image again and inspect the format
image2 = Image.open('opera_house.png')
print(image2.format)'''